In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
from PIL import Image 
import glob
import os

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape( -1, 28, 28, 1)
X_test = X_test.reshape( -1, 28, 28, 1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

# 시각화를 위해 sample 데이터를 준비합니다.
X_sample = X_test[:50]
y_sample = y_test[:50]

CLASS_NAMES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
wandb.login(key = '487e8dc6845db9a1709f5d655dd08945eaf8665f')

wandb: Currently logged in as: kyskyj22 (kyskyj22-aiffel). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [7]:
sweep_config = {
    "name": "sweep_test_core",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

def train():
    default_config = {
        "input" : (28,28,1),
        "filter" : 16,
        "kernel" : (3,3),
        "activation" : "relu",
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }
    
    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(config.filter, config.kernel, activation=config.activation, input_shape=config.input))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback(validation_data = (X_sample, y_sample),
                                        lables = CLASS_NAMES,
                                        predictions = 10,
                                        input_type = "images")])
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

In [13]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'kyskyj22-aiffel',
                       project = 'JellyDeep')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: ay8b9puh
Sweep URL: https://wandb.ai/kyskyj22-aiffel/JellyDeep/sweeps/ay8b9puh


wandb: Agent Starting Run: r3xbllu8 with config:
wandb: 	epoch: 8
wandb: 	learning_rate: 0.08023363550116647


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/8
1875/1875 [==============================] - 50s 3ms/step - loss: 0.7978 - accuracy: 0.7707 - val_loss: 0.6201 - val_accuracy: 0.8030
INFO:tensorflow:Assets written to: /aiffel/aiffel/AIFFEL_quest_cr/MainQuest/Quest06/wandb/run-20241114_025829-r3xbllu8/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/AIFFEL_quest_cr/MainQuest/Quest06/wandb/run-20241114_025829-r3xbllu8/files/model-best)... Done. 0.0s


...

wandb: Adding directory to artifact (/aiffel/aiffel/AIFFEL_quest_cr/MainQuest/Quest06/wandb/run-20241114_030807-m3knrwzy/files/model-best)... Done. 0.0s


Epoch 5/7
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3114 - accuracy: 0.1051 - val_loss: 2.3193 - val_accuracy: 0.0920
Epoch 6/7
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3103 - accuracy: 0.1042 - val_loss: 2.3080 - val_accuracy: 0.0976
INFO:tensorflow:Assets written to: /aiffel/aiffel/AIFFEL_quest_cr/MainQuest/Quest06/wandb/run-20241114_030807-m3knrwzy/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/AIFFEL_quest_cr/MainQuest/Quest06/wandb/run-20241114_030807-m3knrwzy/files/model-best)... Done. 0.0s


Epoch 7/7
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3116 - accuracy: 0.1025 - val_loss: 2.3193 - val_accuracy: 0.0960
157/157 - 0s - loss: 2.3162 - accuracy: 0.0988


Test Accuracy Rate:,▁
Test Error Rate:,▁
accuracy,▆▄▃▁█▆▁
epoch,▁▂▃▅▆▇█
loss,█▂▁▁▂▁▂
val_accuracy,▄█▁▅▁▃▃
val_loss,▅▇▄▂█▁█
Test Accuracy Rate:,9.88
Test Error Rate:,90.12
accuracy,0.10248
best_epoch,5


In [16]:
import os

# 홈 디렉토리 경로 확장
dataset_path = os.path.expanduser('~/aiffel/jellyfish')

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 1504 images belonging to 7 classes.
Found 375 images belonging to 7 classes.


In [14]:
# WandB 초기 설정
wandb.init(
    project="jellyfish-classification",  # 프로젝트 이름
    config={
        "img_size": (128, 128),
        "batch_size": 32,
        "epochs": 20,
        "learning_rate": 0.001,
    }
)
config = wandb.config

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [22]:
# 데이터 전처리
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 크기와 배치 크기 설정
IMG_SIZE = config.img_size
BATCH_SIZE = config.batch_size

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '~/aiffel/jellyfish',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    '~/aiffel/jellyfish',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

FileNotFoundError: [Errno 2] No such file or directory: '~/aiffel/jellyfish'

In [20]:
# 모델 정의
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 해파리 종류가 6종
])

model.compile(optimizer=Adam(learning_rate=config.learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# wandb 콜백 추가 및 모델 학습
from tensorflow.keras.callbacks import EarlyStopping
from wandb.keras import WandbCallback

# EarlyStopping 및 WandB 콜백 설정
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    WandbCallback()
]

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=config.epochs,
    callbacks=callbacks
)

Epoch 1/20


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,6] labels_size=[32,7]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at opt/conda/lib/python3.9/site-packages/wandb/integration/keras/keras.py:174) ]] [Op:__inference_train_function_357591]

Function call stack:
train_function
